# Perceptually Uniform Color Interpolation

This was a small for fun experiment done on a lazy Saturday. Check the original at https://github.com/dreavjr/colorinterp. 

(c̸) 2017 Eduardo Valle. This software is in Public Domain; it is provided "as is" without any warranties. Please check https://github.com/dreavjr/colorinterp/blob/master/LICENSE.md 

## Imports and initializations

In [1]:
import re

import colorlover as cl
from IPython.display import HTML
import numpy as np

num_original = 4
num_interpolated = 10
colors = cl.scales[str(num_original).strip()]['qual']['Set1']
HTML(cl.to_html( colors ))

## Parsing

In [2]:
colors_list = [ re.search('rgb\(([0-9]+),([0-9]+),([0-9]+)\)', c).groups() for c in colors ]
colors_array = np.asarray([ [ float(p) for p in c ] for c in colors_list ])
# DEBUG : forces sRGB correspondence on my hardware
# colors = np.asarray([ [172, 205, 229], [59, 117, 184], [181, 225, 128] ], dtype='float') 
colors_array


array([[ 228.,   26.,   28.],
       [  55.,  126.,  184.],
       [  77.,  175.,   74.],
       [ 152.,   78.,  163.]])

## Conversion from RGB (assumed sRGB) to XYZ
[The formulas copied from Wikipedia article on sRGB.](https://en.wikipedia.org/wiki/SRGB#The_reverse_transformation)

In [3]:
Scale = 255.0
srgb = colors_array/Scale
srgb_thres=0.04045
srgb_linear = np.empty(srgb.shape)
srgb_linear[srgb<=srgb_thres] = (srgb/12.92)[srgb<=srgb_thres]
srgb_linear[srgb>srgb_thres] = np.power(((srgb+0.055)/1.055),2.4)[srgb>srgb_thres]
srgb2xyz = np.asarray([ [ 0.4124, 0.3576, 0.1805 ],
                        [ 0.2126, 0.7152, 0.0722 ],
                        [ 0.0193, 0.1192, 0.9505 ] ])
# xyz = srgb2xyz.dot(srgb_linear[i])
xyz = srgb_linear.dot(srgb2xyz.T)
xyz

array([[ 0.32573904,  0.1731661 ,  0.02724212],
       [ 0.17688132,  0.19194626,  0.48120069],
       [ 0.19626571,  0.32732137,  0.11762073],
       [ 0.22284143,  0.14768522,  0.36326437]])

## Conversion from XYZ to L\*a\*b\*
The formulas copied from [Wikipedia article on L\*a\*b\*](https://en.wikipedia.org/wiki/Lab_color_space#Forward_transformation) and [Bruce Lind Bloom's page](www.brucelindbloom.com/Math.html) on color conversion.

In [4]:
Lab_thresh = 216.0/24389.0
Lab_kappa  = 24389.0/27.0
def xyz2lab(t):
    t2 = np.empty(t.shape)
    cond = t<=Lab_thresh
    t2[cond] = (Lab_kappa*t+16.0)[cond]
    t2[np.logical_not(cond)] = np.power(t,(1.0/3.0))[np.logical_not(cond)]
    return t2

WhiteD65 = np.asarray([ 0.95047, 1.00000, 1.08883 ])
xyz_ratio = xyz/WhiteD65

pre_lab = xyz2lab(xyz_ratio)
ell = 116.0*pre_lab[:,1] - 16.0
a   = 500.0*(pre_lab[:,0]-pre_lab[:,1])
b   = 200.0*(pre_lab[:,1]-pre_lab[:,2])
lab = np.asarray([ell, a, b]).T
lab

array([[ 48.65651299,  71.21070676,  52.98109021],
       [ 50.91413545,  -2.95910047, -36.97247291],
       [ 63.94342979, -49.05118575,  42.58212343],
       [ 45.31550883,  44.02012702, -32.9967354 ]])

# Interpolate colors in the perceptually uniform L\*a\*b\* space

In [5]:
originals = np.arange(ell.shape[0])
interpolated = np.linspace(originals[0], originals[-1], num=num_interpolated)
ell_interp = np.interp(interpolated, originals, ell) 
a_interp = np.interp(interpolated, originals, a) 
b_interp = np.interp(interpolated, originals, b) 

# DEBUG: disables interpolation
# ell_interp = ell
# a_interp = a
# b_interp = b

originals
interpolated
np.asarray([ell_interp, a_interp, b_interp]).T


array([[ 48.65651299,  71.21070676,  52.98109021],
       [ 49.40905381,  46.48743768,  22.99656917],
       [ 50.16159463,  21.76416861,  -6.98795187],
       [ 50.91413545,  -2.95910047, -36.97247291],
       [ 55.25723356, -18.3231289 , -10.45427413],
       [ 59.60033168, -33.68715732,  16.06392465],
       [ 63.94342979, -49.05118575,  42.58212343],
       [ 57.7341228 , -18.02741483,  17.38917049],
       [ 51.52481582,  12.99635609,  -7.80378246],
       [ 45.31550883,  44.02012702, -32.9967354 ]])

## Conversion from L\*a\*b\* to XYZ
[The formulas copied from Wikipedia article on L\*a\*b\*.](https://en.wikipedia.org/wiki/Lab_color_space#Reverse_transformation) and [Bruce Lind Bloom's page](www.brucelindbloom.com/Math.html) on color conversion.

In [6]:
def lab2xyz(t):
    t2 = np.empty(t.shape)
    cond =t<=Lab_thresh
    t2[cond] = ((t-16.0)/Lab_kappa)[cond]
    t2[np.logical_not(cond)]  = np.power(t,3.0)[np.logical_not(cond)]
    return t2

ell_interp_scaled = (ell_interp+16.0)/116.0
a_interp_scaled = a_interp/500.0
b_interp_scaled = b_interp/200.0
pre_x_interp = lab2xyz(ell_interp_scaled+a_interp_scaled)
pre_y_interp = lab2xyz(ell_interp_scaled)
pre_z_interp = lab2xyz(ell_interp_scaled-b_interp_scaled)
pre_xyz_interp = np.asarray([pre_x_interp, pre_y_interp, pre_z_interp]).T
xyz_interp = pre_xyz_interp*WhiteD65
xyz_interp #, xyz


array([[ 0.32573904,  0.1731661 ,  0.02724212],
       [ 0.26935755,  0.17928322,  0.0984861 ],
       [ 0.21988902,  0.18554272,  0.24147296],
       [ 0.17688132,  0.19194626,  0.48120069],
       [ 0.18319378,  0.23179972,  0.32245837],
       [ 0.18965467,  0.27681979,  0.20314101],
       [ 0.19626571,  0.32732137,  0.11762073],
       [ 0.20487495,  0.25682162,  0.17986583],
       [ 0.21373235,  0.19724966,  0.26091937],
       [ 0.22284143,  0.14768522,  0.36326437]])

## Conversion from XYZ to RGB (assumed sRGB)
[The formulas copied from Wikipedia article on sRGB.](https://en.wikipedia.org/wiki/SRGB#The_forward_transformation_.28CIE_XYZ_to_sRGB.29)

In [7]:
xyz2srgb = np.asarray([ [  3.2406, -1.5372, -0.4986 ],
                        [ -0.9689,  1.8758,  0.0415 ],
                        [  0.0557, -0.2040,  1.0570 ] ])
# srgb_linear_interp = xyz2srgb.dot(xyz_interp[i])
srgb_linear_interp = xyz_interp.dot(xyz2srgb.T)
srgb_interp = np.empty(srgb_linear_interp.shape)

srgb_interp[srgb_linear_interp<=0.0031308] = (12.92*srgb_linear_interp)[srgb_linear_interp<=0.0031308]
srgb_interp[srgb_linear_interp>0.0031308] = ((1.055)*np.power(srgb_linear_interp,(1.0/2.4)) - 0.055)[srgb_linear_interp>0.0031308]
srgb_interp[srgb_interp<0.0] = 0.0
srgb_interp[srgb_interp>1.0] = 1.0
srgb_device_interp = np.round(srgb_interp*Scale)
# print(srgb_linear_interp, srgb_interp, srgb_device_interp, colors, sep='\n\n')
srgb_device_interp

array([[ 228.,   26.,   28.],
       [ 195.,   80.,   81.],
       [ 150.,  106.,  132.],
       [  55.,  126.,  184.],
       [  78.,  142.,  150.],
       [  84.,  158.,  114.],
       [  77.,  175.,   74.],
       [ 117.,  147.,  108.],
       [ 139.,  116.,  136.],
       [ 152.,   78.,  163.]])

## Formatting the output

In [8]:
# This --- finally --- is the desired output
colors_interpolated = [ 'rgb(%d,%d,%d)' % tuple(c) for c in srgb_device_interp ]

after_hsl_and_back = cl.to_rgb(cl.to_hsl(colors_interpolated))
print(colors)
HTML(cl.to_html( colors ))

['rgb(228,26,28)', 'rgb(55,126,184)', 'rgb(77,175,74)', 'rgb(152,78,163)']


In [9]:
print(colors_interpolated)
HTML(cl.to_html( colors_interpolated ))

['rgb(228,26,28)', 'rgb(195,80,81)', 'rgb(150,106,132)', 'rgb(55,126,184)', 'rgb(78,142,150)', 'rgb(84,158,114)', 'rgb(77,175,74)', 'rgb(117,147,108)', 'rgb(139,116,136)', 'rgb(152,78,163)']


In [10]:
print(after_hsl_and_back)
HTML(cl.to_html(after_hsl_and_back))

['rgb(230, 25, 29)', 'rgb(195, 80, 82)', 'rgb(149, 106, 131)', 'rgb(55, 126, 185)', 'rgb(78, 143, 151)', 'rgb(83, 157, 112)', 'rgb(77, 176, 74)', 'rgb(117, 147, 108)', 'rgb(139, 116, 136)', 'rgb(151, 78, 162)']
